<a href="https://colab.research.google.com/github/nonotoy/yubimoji_palm/blob/main/yubimoji_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Prepare

#### 0_Install

In [51]:
!pip install tensorflow==2.16.1 scikeras #keras torch==2.0.1

In [52]:
! pip install pydot graphviz

In [53]:
#pip uninstall scikeras keras tensorflow -y

In [54]:
##pip install scikeras==0.13.0 keras==3.2.1 tensorflow==2.16.1 scikit-learn==1.4.2

#### 1_Library

In [55]:
import sklearn
print("scikit-learn version:", sklearn.__version__)

import scikeras
print("scikeras version:", scikeras.__version__)

import keras
print("keras version:", keras.__version__)

import tensorflow
print("tensorflow version:", tensorflow.__version__)

scikit-learn version: 1.4.2
scikeras version: 0.13.0
keras version: 3.3.2
tensorflow version: 2.16.1


In [56]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.test.is_gpu_available())

TensorFlow version: 2.16.1
GPU available: True


In [57]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

import csv

import numpy as np

from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPooling1D, BatchNormalization, Lambda
from keras.regularizers import l2
from keras.utils import to_categorical
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from scikeras.wrappers import KerasClassifier
from sklearn.ensemble import VotingClassifier

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12385778663794766570
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 21991653376
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17334220247690801415
 physical_device_desc: "device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9"
 xla_global_id: 416903419]

#### 2_Common def

In [58]:
# Common parameters
n_sequence = 15  # シーケンス長
n_classes = 76 # 出力クラス (指文字) の数

def setup_pathandparams(mode):

    if mode == 0:
        withPalm = True
        palmNormlized = True

        # 手掌長あり & 手掌長正規化
        dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/point_history/normalised__point_history_normalisedpalm_combined.csv'
        model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/gesture_classifier_0'

        # Parameters
        n_features = 41  # 特徴量の数 (ランドマークx, y 各20列 + 手掌長 1列)

    elif mode == 1:
        withPalm = True
        palmNormlized = False

        # 手掌長あり
        dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/point_history/normalised__point_history_palm_combined.csv'
        model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/gesture_classifier_1'

        # Parameters
        n_features = 41  # 特徴量の数 (ランドマークx, y 各20列 + 手掌長 1列)

    elif mode == 2:
        withPalm = False
        palmNormlized = False

        # 手掌長なし
        dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/point_history/normalised__point_history_combined.csv'
        model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/gesture_classifier_2'

        # Parameters
        n_features = 40  # 特徴量の数 (ランドマークx, y 各20列)

    return withPalm, palmNormlized, dataset, model_save_path, n_features

### Ensemble

In [ ]:
def createLSTMModel():

    # モデル
    model = Sequential([
        Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(n_sequence, n_features)),
        MaxPooling1D(pool_size=2),
        LSTM(512, return_sequences=False, kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(n_classes, activation='softmax')
    ])

    # モデルのコンパイル
    model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


class EnsembleModel(tf.keras.layers.Layer):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = models

    def call(self, inputs):
        predictions = []
        for model in self.models:
            pred = model(inputs)
            predictions.append(pred)

        ensemble_pred = tf.keras.backend.mean(tf.keras.backend.stack(predictions), axis=0)
        return ensemble_pred

    def get_config(self):
        config = super(EnsembleModel, self).get_config()
        config.update({
            'models': [model.get_config() for model in self.models]
        })
        return config

    @classmethod
    def from_config(cls, config):
        models = [tf.keras.Sequential.from_config(model_config) for model_config in config['models']]
        return cls(models)


def create_ensemble_model(models, input_shape):
    input_layer = Input(shape=input_shape)
    ensemble_layer = EnsembleModel(models)(input_layer)
    ensemble_model = Model(inputs=input_layer, outputs=ensemble_layer)
    return ensemble_model


def train_ensemble_models(X_train, y_train, input_shape, n_classes, num_models, epochs, batch_size):
    models = []
    for i in range(num_models):
        model = createLSTMModel()
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
        models.append(model)
    return models


def predict_ensemble(models, X_test):
    predictions = []
    for model in models:
        pred = model.predict(X_test)
        predictions.append(pred)

    ensemble_pred = np.mean(predictions, axis=0)
    return np.argmax(ensemble_pred, axis=1)

In [ ]:
for mode in range(3):

    withPalm, palmNormlized, dataset, model_save_path, n_features = setup_pathandparams(mode)

    # Ground truth label (1st cln)
    y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

    # One-hot encoding of the GT label
    y_dataset = to_categorical(y_dataset, num_classes=n_classes)

    # Training data (2nd cln to the last cln)
    X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, n_features+1))) # (20730, 40) or ,41)

    # サンプル数をシーケンス長で割り切れるように調整
    total_samples = len(X_dataset)
    max_samples = total_samples - total_samples % n_sequence

    # Reshape
    X_dataset = X_dataset[:max_samples]
    y_dataset = y_dataset[:max_samples]
    X_dataset = X_dataset.reshape(-1, n_sequence, n_features)
    y_dataset = y_dataset.reshape(-1, n_sequence, n_classes)

    #ensemble_model = create_ensemble_model((n_sequence, n_features), n_classes)

    # K-fold cross validation
    k = 5
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    precision_scores = []
    recall_scores = []
    f1_scores = []

    print('withPalm: ', withPalm, '| palmNormlized: ', palmNormlized)

    cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=0, save_weights_only=False)
    es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=0)

    for train_index, test_index in kf.split(X_dataset):
        X_train, X_test = X_dataset[train_index], X_dataset[test_index]
        y_train, y_test = y_dataset[train_index], y_dataset[test_index]

        y_train = np.argmax(y_train[:, -1, :], axis=1)
        y_test = np.argmax(y_test[:, -1, :], axis=1)

        models = train_ensemble_models(X_train, y_train, (n_sequence, n_features), n_classes, num_models=5, epochs=200, batch_size=128)
        ensemble_model = create_ensemble_model(models, input_shape=(n_sequence, n_features))

        y_pred = ensemble_model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)

        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    print(f'Precision: {np.mean(precision_scores):.4f} (+/- {np.std(precision_scores):.4f})')
    print(f'Recall: {np.mean(recall_scores):.4f} (+/- {np.std(recall_scores):.4f})')
    print(f'F1 Score: {np.mean(f1_scores):.4f} (+/- {np.std(f1_scores):.4f})')

    ensemble_model.save(model_save_path + '.h5')
    #ensemble_model.save(model_save_path + '.tflite')

    print('--------------------')

NameError: name 'np' is not defined

### No ensemble

In [61]:
#from keras.regularizers import l1_l2, l2
from tensorflow.keras.regularizers import l1_l2, l2

def createLSTMModel():

    regularizer = l2(0.001)

    # ElasticNet正則化の設定
    #l1_ratio = 0.05 # L1正則化の割合 (0 < l1_ratio < 1)
    #regularizer = l1_l2(l1=0.001 * l1_ratio, l2=0.001 * (1-l1_ratio))

    '''model = Sequential([
        Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(n_sequence, n_features)),
        MaxPooling1D(pool_size=2),
        LSTM(512, return_sequences=False, kernel_regularizer=regularizer),
        Dense(512, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        Dropout(0.2),
        Dense(128, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        Dropout(0.2),
        Dense(n_classes, activation='softmax')
    ])'''

    model = Sequential([
        Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(n_sequence, n_features)),
        MaxPooling1D(pool_size=2),
        #LSTM(2048, return_sequences=True, kernel_regularizer=regularizer),
        Bidirectional(LSTM(1024, return_sequences=True, kernel_regularizer=regularizer)),
        Bidirectional(LSTM(512, return_sequences=False, kernel_regularizer=regularizer)),
        #LSTM(1024, return_sequences=True, kernel_regularizer=regularizer),
        #LSTM(512, return_sequences=False, kernel_regularizer=regularizer),
        #Dense(512, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        #Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=regularizer),
        #BatchNormalization(),
        Dropout(0.3),
        Dense(n_classes, activation='softmax')
    ])


    '''model = Sequential([
            Conv1D(filters=256, kernel_size=3, activation='relu', kernel_regularizer=regularizer, bias_regularizer=regularizer, input_shape=(n_sequence, n_features)),
            MaxPooling1D(pool_size=2),
            LSTM(512, return_sequences=True, kernel_regularizer=regularizer, recurrent_regularizer=regularizer, bias_regularizer=regularizer),
            LSTM(256, return_sequences=True, kernel_regularizer=regularizer, recurrent_regularizer=regularizer, bias_regularizer=regularizer),
            LSTM(128, return_sequences=False, kernel_regularizer=regularizer, recurrent_regularizer=regularizer, bias_regularizer=regularizer),
            BatchNormalization(),
            Dropout(0.2),
            Dense(128, activation='relu', kernel_regularizer=regularizer, bias_regularizer=regularizer),
            BatchNormalization(),
            Dropout(0.2),
            Dense(64, activation='relu', kernel_regularizer=regularizer, bias_regularizer=regularizer),
            BatchNormalization(),
            Dropout(0.2),
            Dense(n_classes, activation='softmax')
        ])'''

    model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
y_test_0 = ''
y_test_1 = ''
y_test_2 = ''
X_test_0 = ''
X_test_1 = ''
X_test_2 = ''

for mode in range(3):

    withPalm, palmNormlized, dataset, model_save_path, n_features = setup_pathandparams(mode)

    # Ground truth label (1st cln)
    y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

    # One-hot encoding of the GT label
    y_dataset = to_categorical(y_dataset, num_classes=n_classes)

    # Training data (2nd cln to the last cln)
    X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, n_features+1))) # (20730, 40) or ,41)

    # サンプル数をシーケンス長で割り切れるように調整
    max_samples = len(X_dataset) - len(X_dataset) % n_sequence

    # Reshape
    X_dataset = X_dataset[:max_samples]
    y_dataset = y_dataset[:max_samples]
    X_dataset = X_dataset.reshape(-1, n_sequence, n_features)
    y_dataset = y_dataset.reshape(-1, n_sequence, n_classes)

    print('withPalm: ', withPalm, '| palmNormlized: ', palmNormlized)

    cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path + '_bilstm.keras', verbose=0, save_weights_only=False)
    es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=0)

    X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=42)
    y_train = np.argmax(y_train[:, -1, :], axis=1)
    y_test = np.argmax(y_test[:, -1, :], axis=1)

    if mode == 0:
        X_test_0 = X_test
        y_test_0 = y_test
    elif mode == 1:
        X_test_1 = X_test
        y_test_1 = y_test
    elif mode == 2:
        X_test_2 = X_test
        y_test_2 = y_test

    best_model = None
    best_accuracy = 0

    for i in range(10):

        model = createLSTMModel()

        model.fit(
            X_train,
            y_train,
            epochs=1000,
            batch_size=256,
            validation_data=(X_test, y_test),
            callbacks=[cp_callback, es_callback],
            verbose=0)

        _, accuracy = model.evaluate(X_train, y_train, verbose=0)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

    y_pred = best_model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_pred, axis=1)

    print('--------------------')

    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f'Accuracy:\t{best_accuracy:.2f}')
    print(f'Precision:\t{precision:.2f}')
    print(f'Recall:\t\t{recall:.2f}')
    print(f'F1 Score:\t{f1:.2f}')

    best_model.save(model_save_path + '.h5', overwrite=True)
    #best_model.save(new_model_save_path + '.h5', include_optimizer=False, overwrite=True)

    print('--------------------')

withPalm:  True | palmNormlized:  True


In [50]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

for mode in range(3):
    # load model
    model = load_model('/content/drive/MyDrive/Colab Notebooks/JSL/gesture_classifier_{}.h5'.format(mode))

    # plot model
    file = str('/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/report/model_{}.png'.format(mode))
    plot_model(model=model, show_shapes=True, expand_nested=True, to_file=file)

    # save svg
    SVG(data=model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

predictions = model.predict(X_test_0)
predicted_classes = np.argmax(predictions, axis=1)

cm = confusion_matrix(y_test_0 predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, predicted_classes)
precision = precision_score(y_test, predicted_classes, average='weighted')
recall = recall_score(y_test, predicted_classes, average='weighted')
f1 = f1_score(y_test, predicted_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

### Torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# モデルの定義
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(32)

        # 重みの初期化
        torch.nn.init.xavier_uniform_(self.lstm.weight_ih_l0)
        torch.nn.init.xavier_uniform_(self.lstm.weight_hh_l0)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        torch.nn.init.xavier_uniform_(self.fc3.weight)
        torch.nn.init.xavier_uniform_(self.fc4.weight)
        torch.nn.init.xavier_uniform_(self.fc5.weight)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.relu(self.bn1(self.fc1(out[:, -1, :])))
        out = self.dropout(out)
        out = self.relu(self.bn2(self.fc2(out)))
        out = self.dropout(out)
        out = self.relu(self.bn3(self.fc3(out)))
        out = self.dropout(out)
        out = self.relu(self.bn4(self.fc4(out)))
        out = self.dropout(out)
        out = self.fc5(out)
        return out

def prepare_data(dataset, n_sequence, n_features, n_classes):
    y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

    # 目標ラベルの範囲を確認し、必要に応じて調整
    #assert np.min(y_dataset) >= 0 and np.max(y_dataset) < n_classes, "Invalid target label range"

    y_dataset = torch.from_numpy(y_dataset).long()

    X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, n_features+1)))

    total_samples = len(X_dataset)
    max_samples = total_samples - total_samples % n_sequence

    X_dataset = X_dataset[:max_samples]
    y_dataset = y_dataset[:max_samples]
    X_dataset = X_dataset.reshape(-1, n_sequence, n_features)

    return X_dataset, y_dataset

def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clipping
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return running_loss / len(dataloader), precision, recall, f1


hidden_size = 512
n_sequence = 15
n_classes = 76
num_layers = 2
num_epochs = 200
batch_size = 128
learning_rate = 0.001

# デバイスの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

for mode in range(3):
    if mode == 0:
        withPalm = True
        palmNormlized = True
        dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/point_history/normalised__point_history_normalisedpalm_combined.csv'
        model_save_path = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/palm_normalised_model/gesture_classifier'
        n_features = 41
    elif mode == 1:
        withPalm = True
        palmNormlized = False
        dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/point_history/normalised__point_history_palm_combined.csv'
        model_save_path = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/palm_model/gesture_classifier'
        n_features = 41
    elif mode == 2:
        withPalm = False
        palmNormlized = False
        dataset = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/point_history/normalised__point_history_combined.csv'
        model_save_path = '/content/drive/Othercomputers/My Mac/Documents/2_study/1_修士/3_副研究/yubimoji/base_model/gesture_classifier'
        n_features = 40

    X_dataset, y_dataset = prepare_data(dataset, n_sequence, n_features, n_classes)

    k = 5
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    precision_scores = []
    recall_scores = []
    f1_scores = []

    print('withPalm:', withPalm, '| palmNormlized:', palmNormlized)

    for train_index, test_index in kf.split(X_dataset):
        X_train, X_test = X_dataset[train_index], X_dataset[test_index]
        y_train, y_test = y_dataset[train_index], y_dataset[test_index]

        train_dataset = TensorDataset(torch.from_numpy(X_train).float(), y_train)
        test_dataset = TensorDataset(torch.from_numpy(X_test).float(), y_test)

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        model = LSTMModel(n_features, hidden_size, num_layers, n_classes).to(device)
        #model.config.use_cache = False
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.NAdam(model.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            print(f'Epoch {epoch + 1}/{num_epochs}')
            train_loss = train_model(model, train_dataloader, criterion, optimizer, device)
            test_loss, precision, recall, f1 = evaluate_model(model, test_dataloader, criterion, device)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    print(f'Precision: {np.mean(precision_scores):.4f} (+/- {np.std(precision_scores):.4f})')
    print(f'Recall: {np.mean(recall_scores):.4f} (+/- {np.std(recall_scores):.4f})')
    print(f'F1 Score: {np.mean(f1_scores):.4f} (+/- {np.std(f1_scores):.4f})')

    torch.save(model.state_dict(), model_save_path + '.pth')

    print('--------------------')

<ipython-input-2-b97cd92bd810>:55: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))


withPalm: True | palmNormlized: True
Epoch 1/200


RuntimeError: Expected hidden[0] size (2, 128, 512), got [1, 128, 512]